In [93]:
with open ('input.txt','r',encoding='utf-8') as f:
    text=f.read()

In [94]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [95]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [96]:
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
encode= lambda s: [stoi[c] for c in s ]
decode= lambda l: ''.join([itos[i] for i in l ])

In [97]:
import torch
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [98]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [99]:
torch.manual_seed(1337)
batch_size=4
block_size=8

def get_batch(split):
    # generate a small batch of data of inputs x and targets y 
    data = train_data if split == 'train' else val_data
    max_index = data.shape[0] - block_size - 1  # Ensure the slice for y does not go out of bounds
    ix = torch.randint(0, max_index, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb=get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-------')

print(xb)

inputs:
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
targets:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
-------
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])


In [100]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

vocab_size=65


class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    
    def forward(self,idx,targets=None):

        #idx and targets are both (B,T) tensor of integers  
        logits= self.token_embedding_table(idx)
        if targets is None:
            loss=None
        else:

          
            B,T,C=logits.shape 
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss= F.cross_entropy(logits, targets)

        return logits,loss
    def generate(self,idx,max_new_tokens):

        for _ in range(max_new_tokens):
            #get the predictions
            
            logits,loss=self(idx)
         
           
            logits=logits[:,-1,:]
            
            probs=F.softmax(logits,dim=-1)
            idx_next=torch.multinomial(probs,num_samples=1)
            
            idx=torch.cat((idx,idx_next), dim=1)
            
        return idx
    


m=BigramLanguageModel(vocab_size)
# Initialize the model
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))


# Forward pass through the model
output = m(xb)




Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [101]:
#create a PyTorch optimizer
optimizer=torch.optim.AdamW(m.parameters(),lr=1e-3)

In [102]:
batch_size=32
epoch=10000

for i in range(epoch):
    xb,yb=get_batch(train_data)
    logits,loss=m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    print(loss.item())



4.716053485870361
4.6538286209106445
4.844427585601807
4.714300155639648
4.721835136413574
4.687216758728027
4.6823577880859375
4.715310573577881
4.74955940246582
4.704411506652832
4.609025001525879
4.864656448364258
4.754913330078125
4.685400485992432
4.680613994598389
4.72925329208374
4.632279872894287
4.599748611450195
4.746699810028076
4.806000709533691
4.790671348571777
4.7555108070373535
4.742114543914795
4.644611835479736
4.684349536895752
4.695922374725342
4.673577308654785
4.684942722320557
4.627813816070557
4.722580909729004
4.723783493041992
4.695619106292725
4.687572956085205
4.698641300201416
4.6458516120910645
4.718040943145752
4.658254146575928
4.698106288909912
4.755119800567627
4.692112922668457
4.622318744659424
4.613256454467773
4.718883514404297
4.664824485778809
4.675805568695068
4.674388885498047
4.605911731719971
4.673999786376953
4.640170574188232
4.692752361297607
4.641706466674805
4.6420488357543945
4.6662774085998535
4.7250075340271
4.685265064239502
4.761273

In [103]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))


Iyoteng h hasbe pave pirance
GRO:
Bagathathar's we waserd ith heroangincenonthioneir thoniteay helti


In [104]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x= torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [105]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow=torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev=x[b,:t+1]
        xbow[b,t]=torch.mean(xprev,dim=0)

In [106]:
torch.manual_seed(42)
a=torch.tril(torch.ones(3,3))
b=torch.randint(0,10,(3,2)).float()
a.shape,b.shape

(torch.Size([3, 3]), torch.Size([3, 2]))

In [107]:
c = a @ b
print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [108]:
x.shape

torch.Size([4, 8, 2])

In [109]:
wei = torch.tril(torch.ones(T,T))
wei=wei/wei.sum(1,keepdim=True)
xbow2=wei@x 

In [110]:
# version 3 : use Softmax
tril = torch.tril(torch.ones(T,T))
wei=torch.zeros((T,T))
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,dim=1)
xbow3= wei  @  x
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [111]:
n_embd = 5
block_size=8
token_embedding_table=nn.Embedding(vocab_size,n_embd)
position_embedding_table=nn.Embedding(block_size,n_embd)
tok_emb=token_embedding_table(xb)
lm_head=nn.Linear(n_embd,vocab_size)
pos_emb=position_embedding_table(torch.arange(T))
x=tok_emb+pos_emb
out=lm_head(x)



In [131]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 2,8,5 #2 batch 8 block size series and for each node we have 5 embeddings
x=torch.randn(B,T,C)

#let's see a single Head perform self-attention
head_size=16
key=nn.Linear(C,head_size,bias=False)
query=nn.Linear(C,head_size,bias=False)
value=nn.Linear(C,head_size,bias=False)
k = key(x) # (B,T,16)
q = query(x) # (B,T,16)
wei= q @ k.transpose(-2,-1)* C*-0.5 #(B,T,T)

tril=torch.tril(torch.ones(T,T))
# wei=torch.zeros((T,T))
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,dim=1)

v=value(x)
out= wei  @ v
out.shape

torch.Size([2, 8, 16])

In [138]:
v.shape

torch.Size([2, 8, 16])

In [137]:
wei.shape

torch.Size([2, 8, 8])

In [135]:
out

tensor([[[-2.1872e-02, -6.3075e-02,  8.8923e-03,  4.5167e-03,  1.7818e-02,
          -4.0878e-02,  2.3850e-02,  8.4952e-02, -1.4982e-03,  4.3460e-02,
           8.2206e-03, -3.1750e-02,  1.0573e-02, -2.0470e-02,  1.2159e-02,
          -4.9015e-02],
         [-6.4140e-03, -1.8420e-02,  2.6536e-03,  1.4164e-03,  5.2119e-03,
          -1.1823e-02,  7.0703e-03,  2.4767e-02, -3.0408e-04,  1.2662e-02,
           2.3732e-03, -9.3289e-03,  3.1269e-03, -5.9657e-03,  3.6202e-03,
          -1.4223e-02],
         [-1.1538e-01, -1.6380e-01,  1.4502e-01,  2.2305e-01,  6.2600e-02,
           1.4273e-01,  2.8739e-01,  1.2641e-01,  2.8446e-01,  4.9150e-02,
          -3.9152e-02, -2.0473e-01,  1.1267e-01, -2.8109e-02,  1.8131e-01,
           7.1779e-02],
         [-2.4890e-02, -6.1412e-02, -8.1379e-02, -4.3719e-02, -7.7069e-02,
          -5.6019e-02, -1.7590e-01, -9.9912e-02, -1.2519e-01,  5.2580e-02,
          -2.1185e-02,  6.9835e-02, -4.7551e-03, -8.9060e-02, -7.5121e-02,
          -1.0431e-02],
    

In [134]:
tril[:T,:T]

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [130]:
wei[0]

tensor([[0.0967, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1583, 0.0711, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1173, 0.0033, 0.2313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1181, 0.3241, 0.0497, 0.3773, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1697, 0.0069, 0.2419, 0.0334, 0.0069, 0.0000, 0.0000, 0.0000],
        [0.1891, 0.0026, 0.1866, 0.0394, 0.0027, 0.0872, 0.0000, 0.0000],
        [0.0901, 0.5787, 0.0259, 0.5251, 0.9416, 0.0664, 0.9463, 0.0000],
        [0.0607, 0.0133, 0.2647, 0.0247, 0.0488, 0.8464, 0.0537, 1.0000]],
       grad_fn=<SelectBackward0>)

In [124]:
k=torch.randn(B,T,head_size)
q=torch.randn(B,T,head_size)
wei=q@k.transpose(-2,-1)*head_size**-0.5

In [125]:
k.var()

tensor(1.0024)

In [126]:
q.var()

tensor(0.8937)

In [127]:
wei.var()

tensor(0.8562)

In [75]:
head_size

16

In [73]:
torch.softmax(torch.tensor([0.1,-0.2,0.45,-0.2,0.5])*70,dim=0)

tensor([6.7117e-13, 5.0892e-22, 2.9312e-02, 5.0892e-22, 9.7069e-01])

In [ ]:
 1 hour 21 minutes

In [1]:
import torch 

In [6]:
class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):

        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    
    def __call__(self,x):

        xmean=x.mean(1,keepdim=True)
        xvar=x.var(1,keepdim=True)
        xhat=(x-xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        
        return [self.gamma,self.beta]

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32,100)
x = module(x)
x.shape

torch.Size([32, 100])

In [23]:
x[:1,:].shape,x[:1,:]

(torch.Size([1, 100]),
 tensor([[ 0.1335, -0.1059, -0.3824, -0.9127,  0.5583, -0.0147,  0.8721,  0.0223,
           0.3057,  1.0758, -1.3277, -0.5261,  0.1862, -0.2680, -0.9184,  1.4341,
           1.2485, -0.1724,  0.2337,  0.8822, -1.9837,  0.4317,  1.3804,  0.5251,
           0.0812, -1.5308, -1.1465, -0.3587,  0.3884, -0.8043,  1.4153,  2.3556,
          -0.6721, -0.2789,  0.9252,  0.0769,  0.1121,  1.0434, -1.1406, -0.3239,
          -0.5236, -0.9210,  0.4828, -1.4660, -1.1900,  0.5068, -0.6093, -0.7013,
           1.5317, -0.8056,  1.2509, -0.3025, -1.4813,  1.9701,  2.5984, -1.7063,
           1.3466, -1.4808,  0.7448, -0.2410,  0.7044,  1.4245,  1.4975, -0.4240,
          -0.8357,  0.5316, -0.0882, -0.1007, -0.5135,  0.4055, -0.2847, -1.0579,
           1.8786, -0.5517,  0.1736,  0.6263, -1.4009,  0.8257,  0.0989,  0.1786,
           2.3378, -1.2072,  0.9257,  0.4917, -1.5602, -1.2520,  0.6199, -0.2263,
           0.2902, -0.3654,  0.4152, -0.7706, -0.4390, -0.4562, -0.9948, -0